<a href="https://colab.research.google.com/github/alliec45/mixed_reality_defense/blob/main/GayatriKam_CS4804_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Homework 2, CS4804 Fall 2025, Virginia Tech

### This homework is due on <font color='red'>**November 18$^{th}$, 2025, 11:59 PM ET**</font>. Please submit as a PDF on Gradescope (`File > Print > Save as PDF`). Total: 100 points.

### IMPORTANT: Please save a copy of this notebook to your Google Drive (`File > Save a copy in Drive`). After completing the homework, please share the Colab link below and make sure you grant read access to the instructors by clicking the *Share* button at the top right, entering the email cs4804instructors@gmail.com, and selecting Viewer access. Failure to provide access to the instructors will result in no credit for this homework.

# <font color='red' size='4'>***LINK: *https://colab.research.google.com/drive/1RtNuRnmzk217xW2HYXpH3gaqHvmHlcAc?usp=sharing****</font>

---

##### *How to do this problem set:*

- Some questions require writing Python code and computing results, and the rest of them have written answers. For coding problems, you will have to fill out all code blocks that say `YOUR CODE HERE`.

- For text-based answers, you should replace the text that says `Write your answer here...` with your actual answer.

- There is no penalty for using AI assistance on this homework as long as you fully disclose it in the final cell of this notebook (this includes storing any prompts that you feed to large language models). That said, anyone caught using AI assistance without proper disclosure will receive a zero on the assignment (we have several automatic tools to detect such cases). We are literally allowing you to use it with no limitations, so there is no reason to lie!

---

##### *How to submit this problem set:*
- Write all the answers in this Colab notebook. Once you are finished, generate a PDF via `File -> Print -> Save as PDF` and upload it to Gradescope.
  
- **Important:** check your PDF before you submit to Gradescope to make sure it exported correctly. If Colab gets confused about your syntax, it will sometimes terminate the PDF creation routine early.

- **Important:** on Gradescope, please make sure that you tag each page with the corresponding question(s). This makes it significantly easier for our graders to grade submissions. We may take off points for submissions that are not tagged.

- When creating your final version of the PDF to hand in, please do a fresh restart and execute every cell in order. Then you will be sure it is actually right. One handy way to do this is by clicking `Runtime -> Run All` in the notebook menu.

---

##### *Academic honesty*

- We will audit the Colab notebooks from a set number of students, chosen at random. The audits will check that the code you wrote actually generates the answers in your PDF. If you turn in correct answers on your PDF without code that actually generates those answers, we will consider this a serious case of cheating. See the course page for information on the Graduate Honor Code.

- We will also run automatic checks of Colab notebooks for plagiarism. Copying code from others is also considered a serious case of cheating.

---

# Part 0: TogetherAI and Serper's APIs


In this homework, you will explore how to build a compound AI system that combines a large language model (LLM) with a search engine to improve factual accuracy and access up-to-date information. We will use the Together AI API (https://www.together.ai) to access open-weight LLMs (e.g., `gpt-oss`, `llama`) and the Serper API (https://serper.dev) to perform Google searches.

To get started:

1. Sign up at [Together AI](https://www.together.ai/) and obtain your API key. Please reach out to the instructors at cs4804instructors@gmail.com if you need an API key. This homework should not take more than $1.00 credits.

2. Sign up at [Serper](https://serper.dev/) to obtain your API key and access 2,500 free search queries.

3. Once you have both API keys, enter them into the code provided below.

**Note: If you run out of credits, you may create a new account to receive additional free credits.**

In [ ]:
#@title Setting up Together AI and Serper's APIs


!pip install together

import datetime
import json
import re

import pandas as pd
import pytz
import requests
from together import Together

from google.colab import data_table
data_table.enable_dataframe_formatter()

current_date = datetime.datetime.now(
        pytz.timezone("America/New_York")
    ).strftime("%B %d, %Y")

# Together AI's API key
TOGETHER_API_KEY = "067a795bd26552fb6f27525455695f6b94d34c2eebd144e13c71dd004f533c9e"  # @param {type:"string"}
client = Together(api_key=TOGETHER_API_KEY)

# Serper's API key
SERPER_API_KEY = "9e9b29549c4ed15d7e23aba93eb552daac5f29a8"   # @param {type:"string"}


def call_together_api(prompt, model, temperature, max_tokens):
  response = client.chat.completions.create(
      model=model,
      messages=[
          {
              "role": "system",
              "content": (
                  "You are a helpful assistant. Answer as concisely as"
                    f" possible. Knowledge cutoff: {current_date}."
              ),
          },
          {"role": "user", "content": prompt},
      ],
      max_tokens=max_tokens,
      temperature=temperature,
  )
  return response.choices[0].message.content


def call_google_search(query, top_k_search_results=5):
  url = "https://google.serper.dev/search"

  payload = json.dumps({
    "q": query,
    "location": "United States",
    "hl": "en",
    "num": top_k_search_results,
    "page": 1
  })
  headers = {
    'X-API-KEY': SERPER_API_KEY,
    'Content-Type': 'application/json'
  }

  response = requests.request("POST", url, headers=headers, data=payload)
  return json.loads(response.text)["organic"]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: typer
    Found existing installation: typer 0.20.0
    Uninstalling typer-0.20.0:
      Successfully uninstalled typer-0.20.0


Let's try the APIs!

In [ ]:
prompt = "Write a short poem about Virginia Tech"
model = "meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8"
temperature = 0.0
max_tokens = 512

print(call_together_api(prompt, model, temperature, max_tokens))

In Blacksburg's heart, a spirit bold,
Virginia Tech's legacy unfold.
Hokies rise, with hearts aglow,
In Lane Stadium, their pride does grow.

The Drill, a tradition, strong and free,
Echoes through the Appalachian tree.
Innovation, knowledge, and might,
Shine bright, on this Hokie night.


In [ ]:
query = "best restaurants in Blacksburg"
call_google_search(query, top_k_search_results=5)

[{'title': 'THE 10 BEST Restaurants in Blacksburg ...',
  'link': 'https://www.tripadvisor.com/Restaurants-g57513-Blacksburg_Virginia.html',
  'snippet': '1. The Blacksburg Tavern. 4.5. (85 reviews) · 2. 622 North. 4.1. (240 reviews) · 3. Cabo Fish Taco. 4.4. (554 reviews) · 4. Cellar Restaurant. 4.2. (310 reviews).',
  'position': 1},
 {'title': 'Good food in Blacksburg? : r/VirginiaTech',
  'link': 'https://www.reddit.com/r/VirginiaTech/comments/1fuwx33/good_food_in_blacksburg/',
  'snippet': "Hamro Kitchen and Gaucho's are easily the two best restaurants in Blacksburg. jdubau55. • 1y ago. I like Gaucho's a lot. But, dang, if it's not ...",
  'date': '1 year ago',
  'position': 2},
 {'title': 'Dining',
  'link': 'https://stepintoblacksburg.org/food-drink/dining/',
  'snippet': "From Cajun to vegetarian, ethnic to homestyle, the Blacksburg food scene can't be beat. With plentiful and diverse options, you can easily nosh on local treats.",
  'position': 3},
 {'title': 'Italian, Mediter

# Part 1. Question Answering with Large Language Models (45 points)

In this part of the homework, you will use an LLM to answer questions from a small dataset of 40 examples sampled from `Google`'s `FreshQA` dataset ([Vu et al., 2023](https://arxiv.org/abs/2310.03214)). You will evaluate how well the LLM performs both without and with access to up-to-date information retrieved from a search engine.

Let's start by running the code below to load the dataset!

In [ ]:
tiny_freshqa = [
    {
        "idx": 0,
        "question": "Who won the most recent Super Bowl?",
        "answer": "Philadelphia Eagles"
    },
    {
        "idx": 1,
        "question": "What date is this year's Black Friday?",
        "answer": "November 28, 2025"
    },
    {
        "idx": 2,
        "question": "What date was Mother's Day in the United States this year?",
        "answer": "May 11, 2025"
    },
    {
        "idx": 3,
        "question": "What date was Father's Day in the United States this year?",
        "answer": "June 15, 2025"
    },
    {
        "idx": 4,
        "question": "Which broadcast network was this year's Super Bowl broadcaster?",
        "answer": "Fox and Tubi"
    },
    {
        "idx": 5,
        "question": "What date is Thanksgiving this year?",
        "answer": "November 27, 2025"
    },
    {
        "idx": 6,
        "question": "What date was this year's Labor day?",
        "answer": "September 1, 2025"
    },
    {
        "idx": 7,
        "question": "Who is the current ATP top-ranked men's singles tennis player?",
        "answer": "Carlos Alcaraz"
    },
    {
        "idx": 8,
        "question": "Who is the number 1 ranked female tennis player in the world?",
        "answer": "Aryna Sabalenka"
    },
    {
        "idx": 9,
        "question": "Who is the latest MotoGP World Riders' Champion?",
        "answer": "Marc Márquez"
    },
    {
        "idx": 10,
        "question": "On what date did the Patriots last play the Miami Dolphins?",
        "answer": "September 14, 2025"
    },
    {
        "idx": 11,
        "question": "Who won the latest Nobel Prize in Literature?",
        "answer": "László Krasznahorkai"
    },
    {
        "idx": 12,
        "question": "Who won the latest NBA championship?",
        "answer": "Oklahoma City Thunder"
    },
    {
        "idx": 13,
        "question": "What is the title of Stephen King's most recent novel?",
        "answer": "Never Flinch"
    },
    {
        "idx": 14,
        "question": "How many countries have recognized same-sex marriage?",
        "answer": "38 countries"
    },
    {
        "idx": 15,
        "question": "Who is the starting running back for the San Francisco 49ers?",
        "answer": "Christian McCaffrey"
    },
    {
        "idx": 16,
        "question": "Who won the latest Grammy Award for Best Jazz Performance?",
        "answer": "Samara Joy"
    },
    {
        "idx": 17,
        "question": "What is King Gizzard's most recent studio album?",
        "answer": "Phantom Island"
    },
    {
        "idx": 18,
        "question": "How many accounts have exceeded 100 million followers on Instagram?",
        "answer": "39"
    },
    {
        "idx": 19,
        "question": "What won the latest Academy Award for Best Animated Short Film?",
        "answer": "In the Shadow of the Cypress"
    },
    {
        "idx": 20,
        "question": "Who was the second spouse of the current President of the United States?",
        "answer": "Marla Maples"
    },
    {
        "idx": 21,
        "question": "Who are the latest champions of the North America's oldest professional sports trophy?",
        "answer": "Florida Panthers"
    },
    {
        "idx": 22,
        "question": "In which country was the richest woman in the world born?",
        "answer": "United States"
    },
    {
        "idx": 23,
        "question": "How old is the world's oldest verified living person?",
        "answer": "116 years old"
    },
    {
        "idx": 24,
        "question": "In what country was the most recent Time Person of the Year born?",
        "answer": "United States"
    },
    {
        "idx": 25,
        "question": "What is Brad Pitt's most recent movie as an actor?",
        "answer": "F1"
    },
    {
        "idx": 26,
        "question": "In what city was the person currently acting as the U.S. speaker of house born?",
        "answer": "Shreveport"
    },
    {
        "idx": 27,
        "question": "What country does the Wimbledon women's champion play for?",
        "answer": "Poland"
    },
    {
        "idx": 28,
        "question": "Is one US dollar worth more than 82 Indian rupees?",
        "answer": "Yes"
    },
    {
        "idx": 29,
        "question": "When was the most recent year the world's largest active volcano erupted?",
        "answer": "2022"
    },
    {
        "idx": 30,
        "question": "Who is the most recent Portuguese player to finish in the top three for the Ballon d'Or?",
        "answer": "Vitinha"
    },
    {
        "idx": 31,
        "question": "Who is the owner of the account that has the second most-liked Instagram post?",
        "answer": "Chris Godfrey"
    },
    {
        "idx": 32,
        "question": "How old is the oldest planet known to have life on it?",
        "answer": "4.54 billion years"
    },
    {
        "idx": 33,
        "question": "Which member of The Beatles was born first?",
        "answer": "Ringo Starr"
    },
    {
        "idx": 34,
        "question": "How old is the richest person in the world?",
        "answer": "54 years old"
    },
    {
        "idx": 35,
        "question": "How old is David Beckham's daughter?",
        "answer": "14 years old"
    },
    {
        "idx": 36,
        "question": "Do Tesla and X Corp. have the same CEO?",
        "answer": "No"
    },
    {
        "idx": 37,
        "question": "How many presidents has Peru had in the last 5 years?",
        "answer": "five"
    },
    {
        "idx": 38,
        "question": "Has Virginia Woolf's novel about the Ramsay family entered the public domain in the United States?",
        "answer": "Yes"
    },
    {
        "idx": 39,
        "question": "How many active players have won at least five ATP Finals singles titles?",
        "answer": "one"
    },
]

### Question 1.1 (5 points)

The above dataset contains two types of questions:


1.   **Type 1** (indices 0–19)
2.   **Type 2** (indices 20–39)


Take a look through the questions and reflect on the differences between the two types. What patterns or distinctions do you observe? Consider aspects such as how the questions are phrased, the kind of information needed, or the reasoning involved.

Type 1 questions ask about one specific fact and do not require multi-step reasoning. Further, each of these questions has answers that are subject to change. None of the questions has answers that are guaranteed to be set in stone. Type 1 questions contain phrases like "latest" and "this year." Answers to questions inquiring about this year will have different answers depending on when they are asked. On the other hand, some Type 2 questions have stable, unchanging answers. However, Type 2 questions require multiple pieces of information and a chain reaction of reasoning. These questions are really asking multiple questions in one. For example, the Type 2 question "In which country was the richest woman in the world born?" requires first obtaining the name of the richest woman in the world, then performing an additional search to find out which country she was born in. If this were a Type 1 question, it would be phrased more like 'In which country was Alice Walton born?" Further, the answer to this question is unlikely to change within the next year, as wealth rankings shift relatively slowly, especially for women, often taking generations. Another example is the Type 2 question "Which member of The Beatles was born first?" One search is not enough to answer this question; a search must be made for each member of The Beatles's year of birth, and then the results of each of these queries must be compared to each other to find the minimum. Unlike many Type 1 questions, however, the answer to this question will never change; it is a historical fact. Type 1 questions differ in complexity; take the Type 1 question "What is the title of Stephen King's most recent novel?" for example. To fit the structure of a Type 2 question, it could be phrased as "What is the title of the most recent novel by the author of It?"

### Question 1.2 (15 points)

In this question, you will use the LLM `meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8` to answer all questions _without_ access to up-to-date information, including the current date (`current_date`). You will evaluate each response manually.

Your prompt should instruct the model to answer the question as concisely as possible and return a `JSON` object that contains the following keys:

1.   `main_answer`: a short, direct answer (no more than 10 words)
2.   `explanation`: a brief justification or reasoning


If the model fails to output a `json` object or mixes it with some text, set `main_answer` to "N/A" and assign the entire model response to `explanation`.

Each response should be graded as either **correct** or **incorrect** (including both incorrect answers and cases where the model did not attempt to answer). Your evaluation should focus solely on the correctness of the `main_answer`, while allowing the model to hallucinate in the `explanation`. Note that ill-formed answers are acceptable. However, for date-related questions, the answer must include the exact day, month, and year.

In [ ]:
from aiohttp.client import ClientConnectorCertificateError
model = "meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8"
temperature = 0.0
max_tokens = 512

# Use the function below to extract and load the JSON object from the model's output.
# Feel free to implement your own version if needed.
def extract_json_object(text):
    # Extract content between ```json and ``` using regex
    match = re.search(r"```json\s*(\{.*?\})\s*```", text, re.DOTALL)
    if not match:
        raise ValueError("No valid JSON block found in the text.")

    json_str = match.group(1)
    return json.loads(json_str)


idxes = []
questions = []
correct_answers = []
main_answers = []
explanations = []


# YOUR CODE HERE!
df_freshqa = pd.DataFrame(tiny_freshqa)
for i, row in df_freshqa.iterrows():
  print(f"Processing question {i+1}/40")
  question_id = row['idx']
  question_text = row['question']
  correct_answer = row['answer']

  prompt = f"""Answer the following question concisely and accurately.
  Return ONLY a JSON object wrapped in markdown code blocks, with no other text.

  Format:
  ```json
  {{
    "main_answer": "your short answer (no more than 10 words, include exact day, month, and year if necessary)",
    "explanation": "brief justification or reasoning"
  }}
  ```

  Question: {question_text}"""

  response = call_together_api(prompt, model, temperature, max_tokens)
  try:
    json_response = extract_json_object(response)
    main_answer = json_response['main_answer']
    explanation = json_response['explanation']
  except (ValueError, KeyError) as e:
      # fallback if JSON parsing fails
      print(f"Failed to parse JSON for question {question_id}: {e}")
      main_answer = "N/A"
      explanation = response

  idxes.append(question_id)
  questions.append(question_text)
  correct_answers.append(correct_answer)
  main_answers.append(main_answer)
  explanations.append(explanation)

df_zero_shot = pd.DataFrame({
    'question_id': idxes,
    'question': questions,
    'correct_answer': correct_answers,
    'main_answer': main_answers,
    'explanation': explanations
})

Processing question 1/40
Processing question 2/40
Processing question 3/40
Processing question 4/40
Processing question 5/40
Processing question 6/40
Processing question 7/40
Processing question 8/40
Processing question 9/40
Processing question 10/40
Processing question 11/40
Processing question 12/40
Processing question 13/40
Processing question 14/40
Processing question 15/40
Processing question 16/40
Processing question 17/40
Processing question 18/40
Processing question 19/40
Processing question 20/40
Processing question 21/40
Processing question 22/40
Processing question 23/40
Processing question 24/40
Processing question 25/40
Processing question 26/40
Processing question 27/40
Processing question 28/40
Processing question 29/40
Processing question 30/40
Processing question 31/40
Processing question 32/40
Processing question 33/40
Processing question 34/40
Processing question 35/40
Processing question 36/40
Processing question 37/40
Processing question 38/40
Processing question 3

Now, print out `df_zero_shot` and conduct a human evaluation of the model's responses.

In [ ]:
df_zero_shot

,question_id,question,correct_answer,main_answer,explanation
0,0,Who won the most recent Super Bowl?,Philadelphia Eagles,Kansas City Chiefs won Super Bowl LVIII,"Super Bowl LVIII took place on February 11, 2024"
1,1,What date is this year's Black Friday?,"November 28, 2025","November 29, 2024",Black Friday is the day after US Thanksgiving
2,2,What date was Mother's Day in the United State...,"May 11, 2025","Second Sunday in May, exact date varies by year",Mother's Day is celebrated on the second Sunda...
3,3,What date was Father's Day in the United State...,"June 15, 2025","June 16, 2024",Father's Day is celebrated on the third Sunday...
4,4,Which broadcast network was this year's Super ...,Fox and Tubi,CBS was this year's Super Bowl broadcaster,"Assuming current year is 2024, CBS broadcast S..."
5,5,What date is Thanksgiving this year?,"November 27, 2025",Fourth Thursday of November,"Thanksgiving date varies annually, typically l..."
6,6,What date was this year's Labor day?,"September 1, 2025","First Monday in September, exact date varies b...",Labor Day in the US is celebrated on the first...
7,7,Who is the current ATP top-ranked men's single...,Carlos Alcaraz,"Information not available as of November 19, 2023","The data cutoff is November 19, 2023, and rank..."
8,8,Who is the number 1 ranked female tennis playe...,Aryna Sabalenka,Iga Świątek,"As of my cut-off date, Iga Świątek is ranked n..."
9,9,Who is the latest MotoGP World Riders' Champion?,Marc Márquez,"Francesco Bagnaia, 2023",Won the 2023 MotoGP World Riders' Championship...




```
# This is formatted as code
```

Now, report the model's accuracy based on your manual evaluation. Additionally, provide a brief comment on the model's overall performance.

The model correctly answered questions 15, 16, 24, 29, 32, and 28. 6/40 correctly answered questions translates to 15% accuracy. The model's performance is poor due to a lack of up-to-date information. The questions answered correctly can be attributed to the fact that the answer has not changed in the past few years. You can gauge from the model's responses that the most recent data the model is using is from around 2023 to early 2024. However, it is doubtful that it has access to information from late 2024, as it was able to pull the date for Father's Day and Black Friday of 2024, which can be determined from the beginning of the year, but it did not recognize the presidential switch that occurred later that year. It is reasonable to state that the zero-shot approach is heavily limited for time-sensitive questions without access to current information. It did not only struggle due to a lack of up-to-date information, however. It also struggled with questions that rely on more complex reasoning, like question 33 regarding The Beatles. Regardless of whether the model is accessing curent data, it should have been able to retrieve all birth years and compare them accurately, but it failed to. This suggests that the model can see improved performance before we even feed it current information if we improve its reasoning abilities.

### Question 1.3 (25 points)

Now, revise your previous prompt to use few-shot prompting with `Least-to-Most Prompting` ([Zhou et al., 2022](https://arxiv.org/abs/2205.10625)). This approach involves breaking down the original question into a sequence of simpler subquestions, answering each one step by step, and then combining the answers to produce the final response.


Here are a few sample questions that you can use as demonstration examples. You are also welcome to use your own examples. However, you should still test on the complete test set of 40 examples provided above.

```
[
  {
      "idx": 40,
      "question": "Which state is the 8th president of the United States buried in?",
      "answer": "New York"
  },
  {
      "idx": 41,
      "question": "What is the main character's name in the last novel Jane Austen published while alive?",
      "answer": "Emma"
  },
]
```

In [ ]:
model = "meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8"
temperature = 0.0
max_tokens = 512

def least_to_most_prompting(df):
  idxes = []
  questions = []
  correct_answers = []
  main_answers = []
  explanations = []

  # YOUR CODE HERE!
  total = len(df)

  for i, row in df.iterrows():
      print(f"Processing question {i+1}/{total}")
      question_id = row['idx']
      question_text = row['question']
      correct_answer = row['answer']

      # prompt to break down the question into subquestions
      breakdown_prompt = f"""Break down the following question into a sequence of simpler subquestions that need to be answered step-by-step.
      Question: {question_text}

      Return ONLY a JSON object with the subquestions:
      ```json
      {{
        "subquestions": ["subquestion 1", "subquestion 2", "..."]
      }}
      ```
      """
      breakdown_response = call_together_api(breakdown_prompt, model, temperature, max_tokens)

      try:
          subquestions_json = extract_json_object(breakdown_response)
          subquestions = subquestions_json['subquestions']
      except:
          # if all fails, revert back to original question
          subquestions = [question_text]

      # prompt to answer subquestions and apply combined knowledge to generate final response
      answering_prompt = f"""Answer the following question by first answering each subquestion step-by-step, then combining the answers.
      Original Question: {question_text}

      Subquestions to answer:
      {chr(10).join([f"{idx+1}. {sq}" for idx, sq in enumerate(subquestions)])}

      Answer each subquestion, then provide the final answer only. Return ONLY a JSON object:
      ```json
      {{
        "main_answer": "your final short answer (no more than 10 words, include exact day, month, and year if necessary)",
        "explanation": "step-by-step reasoning showing how you answered each subquestion"
      }}
      ```
      """
      response = call_together_api(answering_prompt, model, temperature, max_tokens)

      try:
          json_response = extract_json_object(response)
          main_answer = json_response['main_answer']
          explanation = json_response['explanation']
      except (ValueError, KeyError):
          try:
              # parse final response more flexibly
              json_match = re.search(r'\{.*?"main_answer".*?"explanation".*?\}', response, re.DOTALL)
              if json_match:
                  json_response = json.loads(json_match.group(0))
                  main_answer = json_response['main_answer']
                  explanation = json_response['explanation']
              else:
                  raise ValueError("No JSON found")
          except:
              # final fallback
              main_answer = "N/A"
              explanation = response

      idxes.append(question_id)
      questions.append(question_text)
      correct_answers.append(correct_answer)
      main_answers.append(main_answer)
      explanations.append(explanation)

  return pd.DataFrame({
        'question_id': idxes,
        'question': questions,
        'correct_answer': correct_answers,
        'main_answer': main_answers,
        'explanation': explanations,
  })

In [ ]:
demo_examples = [
  {
      "idx": 40,
      "question": "Which state is the 8th president of the United States buried in?",
      "answer": "New York"
  },
  {
      "idx": 41,
      "question": "What is the main character's name in the last novel Jane Austen published while alive?",
      "answer": "Emma"
  },
]
df_demo = pd.DataFrame(demo_examples)
df_demo_results = least_to_most_prompting(df_demo)
df_demo_results

Processing question 1/2
Processing question 2/2


,question_id,question,correct_answer,main_answer,explanation
0,40,Which state is the 8th president of the United...,New York,New York,The 8th president is Martin Van Buren. He is b...
1,41,What is the main character's name in the last ...,Emma,Anne Elliot,1. Jane Austen was an English novelist known f...


Now, print out `df_least_to_most` and conduct a human evaluation of the model's responses.

In [ ]:
df_least_to_most = least_to_most_prompting(df_freshqa)
df_least_to_most

Processing question 1/40
Processing question 2/40
Processing question 3/40
Processing question 4/40
Processing question 5/40
Processing question 6/40
Processing question 7/40
Processing question 8/40
Processing question 9/40
Processing question 10/40
Processing question 11/40
Processing question 12/40
Processing question 13/40
Processing question 14/40
Processing question 15/40
Processing question 16/40
Processing question 17/40
Processing question 18/40
Processing question 19/40
Processing question 20/40
Processing question 21/40
Processing question 22/40
Processing question 23/40
Processing question 24/40
Processing question 25/40
Processing question 26/40
Processing question 27/40
Processing question 28/40
Processing question 29/40
Processing question 30/40
Processing question 31/40
Processing question 32/40
Processing question 33/40
Processing question 34/40
Processing question 35/40
Processing question 36/40
Processing question 37/40
Processing question 38/40
Processing question 3

,question_id,question,correct_answer,main_answer,explanation
0,0,Who won the most recent Super Bowl?,Philadelphia Eagles,"Kansas City Chiefs won on February 11, 2024",1. The Super Bowl is the NFL's annual champion...
1,1,What date is this year's Black Friday?,"November 28, 2025","November 24, 2023",1. Black Friday is a major shopping day. 2. It...
2,2,What date was Mother's Day in the United State...,"May 11, 2025","May 14, 2023",1. Assumed 'this year' refers to 2023 due to l...
3,3,What date was Father's Day in the United State...,"June 15, 2025","June 16, 2024",1. 'This year' is assumed to be 2024 for calcu...
4,4,Which broadcast network was this year's Super ...,Fox and Tubi,CBS broadcast the 2024 Super Bowl,1. The current year is 2024. 2. The Super Bowl...
5,5,What date is Thanksgiving this year?,"November 27, 2025","November 28, 2024",1. Thanksgiving is in November. 2. It's on a T...
6,6,What date was this year's Labor day?,"September 1, 2025","September 4, 2023",1. 'This year' is assumed as 2023 due to lack ...
7,7,Who is the current ATP top-ranked men's single...,Carlos Alcaraz,Check ATP website for latest ranking,1. The source for current ATP rankings is the ...
8,8,Who is the number 1 ranked female tennis playe...,Aryna Sabalenka,"Iga Świątek on November 19, 2023","1. Assumed current date: November 19, 2023. 2...."
9,9,Who is the latest MotoGP World Riders' Champion?,Marc Márquez,Francesco Bagnaia is the 2022 MotoGP World Cha...,1. MotoGP is the top class of Grand Prix motor...


Now, report the model's accuracy based on your manual evaluation. Additionally, provide a brief comment on the model's overall performance. Do you find the `Least-to-Most Prompting` approach more effective for Type 1 or Type 2 questions? Explain why.

<font color='red'>**Note: To receive full credit, you should show at least a 7% improvement over the baseline results from Question 1.2.**</font>

The Least-to-Most Prompting approach produced correct answers for questions 15, 16, 24, 26, 28, 29, 32, 33, 37, 38, and 39. At 11/40 correct answers, this approach achieved 27.5% accuracy. This is almost double the accuracy metric determined for zero-shot prompting. However, this improvement in accuracy is primarily attributable to enhanced performance on Type 2 questions. The above run showed 0% improvement for Type 1 questions, as they require the current date to be included in the prompt and access to up-to-date information, not advanced reasoning skills. The results shown above reflect one run of the model. In other runs on the FreshQA dataset, Type 1 questions showed minimal improvement, with at most one additional correct answer. Questions 3 and 6 were occasionally answered correctly in separate runs. These correct answers, however, were exceptional cases where the model assumed the current year is 2025, whereas for most questions, the model defaulted to assuming the year was 2023 or 2024. On the other hand, among Type 2 questions, 5 additional correct answers were observed. Type 2 questions are more often based on reasoning rather than year-sensitive, up-to-date information. Many of them were answered incorrectly with zero-shot prompting not due to the lack of current data, but due to the model's lack of reasoning ability. Many of the Type 2 answers are set in stone and will not change in the future. Therefore, improving the model's ability to decompose questions into subquestions and reason through them step-by-step was effective in boosting accuracy for Type 2 questions.

# Part 2. Search-augmented Large Language Models (35 points)

In this part of the homework, you will explore how combining a large language model with a search engine can improve factual accuracy.

### Question 2.1 (35 points)

In this question, you will implement a simplified version of the `FreshPrompt` method from [Vu et al. (2023)]((https://arxiv.org/abs/2310.03214)). For each question, you will call `call_google_search(question)` to retrieve the top `k` search results (e.g., `k = 5`) and append them to the input. In the few-shot demonstrations, you will include examples that show how to reason over the retrieved search results to arrive at the correct answer. For the demonstration examples, you may reuse the examples from Question 1.3 or create your own.

Your few-shot prompt should follow the format below:

```
// general instructions
Today is {current_date}. ....


question: <demo_question_1>

// list of k search results for <demo_question_1>
title:
date:
snippet:

title:
date:
snippet:

title:
date:
snippet:

....

reasoning: <reasoning_for_demo_question_1>
answer: <answer_for_demo_question_1>


question: <demo_question_2>

// list of k search results for <demo_question_2>
title:
date:
snippet:

title:
date:
snippet:

title:
date:
snippet:

....

reasoning: <reasoning_for_demo_question_2>
answer: <answer_for_demo_question_2>


question: <actual_question>

// list of k search results for <actual_question>
title:
date:
snippet:

title:
date:
snippet:

title:
date:
snippet:

....

reasoning:
```

You can begin the reasoning with a statement such as: "As of today, {current_date}, the most up-to-date and relevant information is as follows."

In [63]:
model = "meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8"
temperature = 0.0
max_tokens = 512

# Format search results
def format_search_results(question, top_k_search_results=5):
  search_results = call_google_search(question, top_k_search_results)
  output = ""

  for k in range(len(search_results)):
    if 'title' in search_results[k]:
      title = search_results[k]['title']
    else:
      title = "N/A"
    output += f"title: {title}\n"

    if 'date' in search_results[k]:
      date = search_results[k]['date']
    else:
      date = "N/A"
    output += f"date: {date}\n"

    if 'snippet' in search_results[k]:
      snippet = search_results[k]['snippet']
    else:
      snippet = "N/A"
    output += f"snippet: {snippet}\n"
  return output


idxes = []
questions = []
correct_answers = []
main_answers = []
explanations = []

# YOUR CODE HERE!
def generate_few_shot_prompt(actual_question, current_date):
  demo1_results = format_search_results("Which state is the 8th president of the United States buried in?", 5)
  demo2_results = format_search_results("What is the main character's name in the last novel Jane Austen published while alive?", 5)
  actual_results = format_search_results(actual_question, 5)

  prompt = f"""Today is {current_date}. Use the provided search results to evaluate and answer questions with the most accurate, up-to-date information.

  INSTRUCTIONS:
  - Analyze the search results carefully
  - Provide reasoning that references the search results
  - End with "answer:" followed by your final answer on a new line
  - For date questions, include the exact day, month, and year (e.g., "May 11, 2025")
  - Keep answers concise (no more than 10 words)

  question: Which state is the 8th president of the United States buried in?

  {demo1_results}

  reasoning: As of today, {current_date}, the most up-to-date and relevant information unanimously confirms that Martin Van Buren, the 8th President of the United States, is buried in the Kinderhook Reformed Church Cemetery in Kinderhook, New York.
  answer: New York


  question: What is the main character's name in the last novel Jane Austen published while alive?

  {demo2_results}

  reasoning: As of today, {current_date}, the most up-to-date and relevant information unanimously confirms that Emma was Jane Austen's last novel published during her lifetime in 1815. The main character in Emma is Emma Woodhouse.
  answer: Emma


  question: {actual_question}

  {actual_results}

  reasoning: As of today, {current_date}, the most up-to-date and relevant information is as follows:"""
  return prompt

for i, row in df_freshqa.iterrows():
    print(f"Processing question {i+1}/40")
    question_id = row['idx']
    question_text = row['question']
    correct_answer = row['answer']

    prompt = generate_few_shot_prompt(question_text, current_date)

    response = call_together_api(prompt, model, temperature, max_tokens)

    response_lower = response.lower()
    if "answer:" in response_lower:
      parts = response.split("answer:", 1)
      if len(parts) > 1:
        explanation = parts[0].strip()
        main_answer = parts[1].strip()
      else:
          # if split doesn't work
          explanation = response
          main_answer = "N/A"
    # fallback: look for "The answer is X" at the end
    elif "the answer is:" in response_lower:
        parts = response.rsplit("The answer is:", 1)
        if len(parts) > 1:
          explanation = parts[0].strip()
          main_answer = parts[1].strip().rstrip('.')
        else:
          # if split doesn't work
          explanation = response
          main_answer = "N/A"
    elif "the answer is" in response_lower:
        parts = response.rsplit("The answer is", 1)
        if len(parts) > 1:
          explanation = parts[0].strip()
          main_answer = parts[1].strip().rstrip('.')
        else:
          # if split doesn't work
          explanation = response
          main_answer = "N/A"
    else:
        explanation = response
        main_answer = "N/A"

    idxes.append(question_id)
    questions.append(question_text)
    correct_answers.append(correct_answer)
    main_answers.append(main_answer)
    explanations.append(explanation)


df_freshprompt = pd.DataFrame({
    'question_id': idxes,
    'question': questions,
    'correct_answer': correct_answers,
    'main_answer': main_answers,
    'explanation': explanations,
})

Processing question 1/40
Processing question 2/40
Processing question 3/40
Processing question 4/40
Processing question 5/40
Processing question 6/40
Processing question 7/40
Processing question 8/40
Processing question 9/40
Processing question 10/40
Processing question 11/40
Processing question 12/40
Processing question 13/40
Processing question 14/40
Processing question 15/40
Processing question 16/40
Processing question 17/40
Processing question 18/40
Processing question 19/40
Processing question 20/40
Processing question 21/40
Processing question 22/40
Processing question 23/40
Processing question 24/40
Processing question 25/40
Processing question 26/40
Processing question 27/40
Processing question 28/40
Processing question 29/40
Processing question 30/40
Processing question 31/40
Processing question 32/40
Processing question 33/40
Processing question 34/40
Processing question 35/40
Processing question 36/40
Processing question 37/40
Processing question 38/40
Processing question 3

In [ ]:
# get search results for demo questions for use in prompt
demo1_search = format_search_results("Which state is the 8th president of the United States buried in?", top_k_search_results=5)
print(demo1_search)

demo2_search = format_search_results("What is the main character's name in the last novel Jane Austen published while alive?", top_k_search_results=5)
print(demo2_search)

title: Martin Van Buren
date: N/A
snippet: Kinderhook, New York, U.S.. Resting place, Kinderhook Reformed Church Cemetery. Political party. Democratic-Republican (1799–1825); Democratic ( ...
title: Martin Van Buren burial site in New York
date: 3 months ago
snippet: Kinderhook Reformed Church Cemetery Kinderhook,NY 8th President of the US, Martin Van Buren Albany Rural Cemetery Albany,NY 21st President of ...
title: Frequently Asked Questions - Martin Van Buren ...
date: N/A
snippet: Where is Martin Van Buren buried? Van Buren is buried in the local Dutch Reformed cemetery in the Village of Kinderhook, about a 5 minute drive from the park.
title: Grave of 8th US president
date: Jul 26, 2025
snippet: A memorial and grave of Martin Van Buren the 8th President of the United States of America.
title: Gravesite of Martin Van Buren - 8th U.S. President
date: 11 months ago
snippet: Gravesite of Martin Van Buren - 8th U.S. President - Kinderhook NY #cemetery #president. 148 views · 11 months 

Now, print out `df_freshprompt` and conduct a human evaluation of the model's responses.

In [64]:
df_freshprompt

,question_id,question,correct_answer,main_answer,explanation
0,0,Who won the most recent Super Bowl?,Philadelphia Eagles,Philadelphia Eagles,The most recent Super Bowl winner is the Phila...
1,1,What date is this year's Black Friday?,"November 28, 2025","November 28, 2025","As of today, November 19, 2025, the most up-to..."
2,2,What date was Mother's Day in the United State...,"May 11, 2025","May 11, 2025",The search results provide information about M...
3,3,What date was Father's Day in the United State...,"June 15, 2025","June 15, 2025",The search results indicate that Father's Day ...
4,4,Which broadcast network was this year's Super ...,Fox and Tubi,FOX,The search results indicate that the Super Bow...
5,5,What date is Thanksgiving this year?,"November 27, 2025","November 27, 2025",The search results consistently indicate that ...
6,6,What date was this year's Labor day?,"September 1, 2025","September 1, 2025","Labor Day 2025 was on September 1, 2025.\nThe ..."
7,7,Who is the current ATP top-ranked men's single...,Carlos Alcaraz,Carlos Alcaraz,The most recent information available indicate...
8,8,Who is the number 1 ranked female tennis playe...,Aryna Sabalenka,Aryna Sabalenka,The WTA Singles World Rankings from November 1...
9,9,Who is the latest MotoGP World Riders' Champion?,Marc Márquez,Marc Márquez,The search results indicate that Marc Márquez ...


Now, report the model's accuracy based on your manual evaluation. Additionally, provide a brief comment on the model's overall performance. Do you find the `FreshPrompt` approach effective for Type 2 questions? Explain why.

<font color='red'>**Note: To receive full credit, you should show at least a 40% improvement over the baseline results from Question 1.2.**</font>

This model correctly answered 18/20 of the Type 1 questions and 16/20 of the Type 2 questions. This translates to an 85% accuracy rate. This represents a 70 percentage point improvement over the 15% baseline from Question 1.2, and 57.5 percentage point improvement from the approach from Question 1.3. Though developing reasoning skills showed improvement for Type 2 questions, this is the first approach that produced significant improvement in Type 1 question results. The improvement was not as significant for Type 2 questions, however, because this set of questions requires breaking down an overarching question into multiple parts. Getting near perfect accuracy for Type 2 questions would likely require multiple Google searches on a series of topics, one for each subquestion identified in the decomposition process. FreshPrompt worked so well for Type 1 questions because it addresses the exact issues that were preventing correct answers. By providing access to current information through search results and explicitly establishing today's date, FreshPrompt directly addressed the knowledge cutoff limitation.

# Part 3. LLM-as-a-Judge (20 points)

In this part of the homework, you will implement an LLM-as-a-Judge model and compare its evaluations to your own.

### Question 3.1 (20 points)

In this question, you will use the LLM `meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8` to judge whether a given prediction matches the correct answer for a question. For each example, provide the model with the question, the correct answer, and the predicted answer. Note that ill-formed answers are acceptable. However, for date-related questions, the answer must include the exact day, month, and year. The model should output only one word: either `correct` or `incorrect`.

Wrap your implementation in a function `call_llm_as_a_judge()` that takes a DataFrame as input (e.g., `df_freshprompt`) and returns two outputs:

1. A DataFrame that includes each original question, its correct answer, the model's prediction, and the LLM's judgment ("correct" or "incorrect") for that prediction.
2. The overall accuracy of the judgments.


Your prompt should follow the format below:

```
// general instructions


question: <question>
correct_answer: <correct_answer>
prediction: <prediction>
judgment:
```

In [65]:
model = "meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8"
temperature = 0.0
max_tokens = 512

# YOUR CODE HERE!
def call_llm_as_a_judge(df):
  judgments = []

  for i, row in df.iterrows():
    question = row['question']
    correct_answer = row['correct_answer']
    prediction = row['main_answer']

    prompt = f"""You are evaluating whether a prediction matches the correct answer to a question.
    INSTRUCTIONS:
    - Ill-formed answers are acceptable if they convey a correct answer
    - For date-related questions, the prediction must include the exact day, month, and year
    - You should only output one word: either "correct" or "incorrect"

    question: {question}
    correct_answer: {correct_answer}
    prediction: {prediction}
    judgment:"""

    response = call_together_api(prompt, model, temperature, max_tokens)

    response_lower = response.strip().lower()
    if "correct" in response_lower and "incorrect" not in response_lower:
        judgment = "correct"
    elif "incorrect" in response_lower:
        judgment = "incorrect"
    else:
        judgment = "incorrect"

    judgments.append(judgment)

  judgment_df = df.copy()
  judgment_df['judgement'] = judgments

  correct_count = judgments.count('correct')
  total = len(judgments)
  accuracy = (correct_count / total) * 100 if total > 0 else 0

  return judgment_df, accuracy

# Uncomment and run the code below after implementing the call_llm_as_a_judge() function
df_judge_zero_shot, accuracy_zero_shot = call_llm_as_a_judge(df_zero_shot)
print(f"zero_shot accuracy: {accuracy_zero_shot:.2f}%")

df_judge_least_to_most, accuracy_least_to_most = call_llm_as_a_judge(df_least_to_most)
print(f"least_to_most accuracy: {accuracy_least_to_most:.2f}%")

df_judge_freshprompt, accuracy_freshprompt = call_llm_as_a_judge(df_freshprompt)
print(f"freshprompt accuracy: {accuracy_freshprompt:.2f}%")

zero_shot accuracy: 27.50%
least_to_most accuracy: 32.50%
freshprompt accuracy: 80.00%


Now, compare the LLM-as-a-Judge's evaluations to your own judgments. Identify any disagreements and briefly discuss possible reasons (e.g., ambiguity in the answer, formatting differences, etc.). Provide a short comment on whether the model's judgments appear reliable overall.


In some cases, I am a more lenian judge than the LLM. For freshprompt's accuracy, I overshot the LLM's judgement by 5%. However, I was a harsher grader for zero shot and least to most accuracy. However, all the accuracy metrics returned by the LLM are in the ballpark of mine, so I would say that the LLM is a reliable grader. Through tweaking the prompt, I can control what factors it is strict with and where I'd like it to be leniant as well. I can add extra stipulations and rules under INSTRUCTIONS to have it grade in a way that aligns with my criteria.



# AI Disclosure

*   Did you use any AI assistance to complete this homework? If so, please also specify what AI you used.
    * Sonnet 4.5


---
*(only complete the below questions if you answered yes above)*

*   If you used a large language model to assist you, please paste *all* of the prompts that you used below. Add a separate bullet for each prompt, and specify which problem is associated with which prompt.

Question 1.1 - Zero-Shot Prompting
* Can you assist me with the assignment? Walk me through it step by step, giving me syntax clues but not code for each step. [uploaded notebook file]
* [provided tiny_freshqa data structure] What differences do you see in questions 0-19 vs 20-39? They are both questions that require up to date information, aren't they?
* Fill in the blank, rephrase that last question, but don't rewrite any of my other sentences. Flag the sentences that have glaring issues though: [provided paragraph about Type 1 and Type 2 questions]
* What converted freshqa into a df? I thought it was just a list of dictionaries
* Why don't I tell the model to provide the main answer then provide a short explanation after a colon, then I will separate main answer from explanation using the colon in between?
* Should I be concerned this is taking longer than a minute with a T4 GPU
* My code is not returning a json object is it? [Your prompt should instruct the model to return a JSON object with main_answer and explanation keys]
* These are the fail safe instructions: [If the model fails to output a json object, set main_answer to "N/A" and assign the entire model response to explanation]
* [provided results table showing N/A answers] Why did these fail to parse?

Question 1.2 - Least-to-Most Prompting

* How can I rephrase my prompt to utilize Least-to-Most prompting? [provided current zero-shot prompt and instructions about breaking down questions into subquestions]
* Why am I having issues with this again? [provided same parsing failure example and current code]
* [provided result] Why did these fail to parse?
* Polish this, but don't return a paragraph to me. Instead, tell me individual sentences that need fixing: [provided paragraph about Least-to-Most results]

Question 1.3 - Demo Examples

* Circling back to question 1.3, I never used the demo examples to walk through the process. I should do that. [provided demo examples with idx 40 and 41] In a separate cell, apply the same process to these examples [provided full least-to-most code]
* Or instead do I put this inside a function that takes a dataframe as a parameter so I can run it on both df_demo and the big dataset: [provided loop code]
* Why does this answer have nothing to do with the reasoning? [provided result showing "Anne Elliot" with reasoning pointing to "Emma"]

Question 2.1 - FreshPrompt

* Why do I hardcode this into my prompt? [provided search results for both demo questions]
* [provided generate_few_shot_prompt function] Where can I indicate for dates I need month, day, year. Also, I want the answers to be in a similar format for extraction purposes. Where can I indicate that?

Question 3.1 - LLM-as-a-Judge

* [provided instructions about implementing LLM-as-a-Judge and empty code cell] Walk me through this step by step, but do not provide me chunks of code. Check my code before moving on to the next step
* [provided partial function with loop structure and prompt] [showed updated code with prompt added]

*   **Free response**: For each problem for which you used assistance, describe your overall experience with the AI. How helpful was it? Did it just directly give you a good answer, or did you have to edit it? Was its output ever obviously wrong or irrelevant? Did you use it to get the answer or check your own answer?
    * In the beginning, when it was not bombarded with information, I got good results on the first try. However, as I went on and it was overloaded with instructions on many different types of questions, it started to become confused. It was at these points that I had to instruct it to slow down and help me step by step.
    * I emphasized that I did not want to be given huge chunks of code. I wanted to work through this project myself. However, I needed syntax hints, and I know from previously working with Claude that it would be very good at breaking down a problem and providing me with only hints and not code.
    * It was especially helpful with debugging JSON errors. There were many times I was getting an excessive number of N/As instead of answers, and it helped me make my parsing code more leniant.
